<a href="https://colab.research.google.com/github/Kzis/ml-programming-python/blob/master/Homework/HW-Click%20Through%20Rate%20Prediction/CTR_Online_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle


In [2]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"driftkao","key":"3567a49504e637a04e8bb1e495e63c45"}'}

In [3]:
!ls -a

.  ..  .config	kaggle.json  sample_data


In [0]:
! mkdir ~/.kaggle


In [0]:
! cp kaggle.json ~/.kaggle/

In [0]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c avazu-ctr-prediction

100% 1.04G/1.04G [00:09<00:00, 31.7MB/s]

 92% 109M/118M [00:03<00:00, 38.7MB/s]
100% 118M/118M [00:03<00:00, 39.6MB/s]
 93% 31.0M/33.2M [00:00<00:00, 46.1MB/s]
100% 33.2M/33.2M [00:00<00:00, 74.8MB/s]


In [0]:
!gunzip test.gz

In [0]:
!gunzip train.gz

In [0]:
import numpy as np
import csv
from sklearn.metrics import (roc_auc_score, confusion_matrix,
                             classification_report)
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
import timeit

In [0]:
def read_ad_click_data(n, offset=0):
    X_dict, y = [], []
    with open('/content/train', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for i in range(offset):
            next(reader)
        i = 0
        for row in reader:
            i += 1
            y.append(int(row['click']))
            del row['click'], row['id'], row['hour'], row['device_id']
            del row['device_ip']
            X_dict.append(row)
            if i >= n:
                break
    return X_dict, y

In [0]:
def get_ad_click_data(n=100000):
    X_dict, y = [], []
    with open('/content/train', 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        i = 0
        for row in reader:
            i += 1
            y.append(int(row['click']))
            del row['click'], row['id'], row['hour'], row['device_id']
            del row['device_ip']
            X_dict.append(row)
            if i >= n:
                yield (X_dict, y)
                X_dict, y = [], []

In [0]:
n = 100000
X_dict_train, y_train = read_ad_click_data(n)
dict_one_hot_encoder = DictVectorizer(sparse=False)
X_train = dict_one_hot_encoder.fit_transform(X_dict_train)

In [15]:
# Online learning

clf = SGDClassifier(loss='log', penalty=None, fit_intercept=True,
                    learning_rate='constant', eta0=0.01)


start_time = timeit.default_timer()

# there are 40428968 labelled samples, use the first twenty 100k samples
# for training, and the next 100k for testing
data = get_ad_click_data(100000)
for i in range(20):
    print(f'Iteration: {i}')
    X_dict_train, y_train_every_100k = next(data)
    X_train_every_100k = dict_one_hot_encoder.transform(X_dict_train)
    clf.partial_fit(X_train_every_100k, y_train_every_100k, classes=[0, 1])


print(f"--- {timeit.default_timer() - start_time:.3f} seconds ---")

X_dict_test, y_test_next10k = read_ad_click_data(10000, (i + 1) * 200000)
X_test_next10k = dict_one_hot_encoder.transform(X_dict_test)


predictions = clf.predict_proba(X_test_next10k)[:, 1]
roc = roc_auc_score(y_test_next10k, predictions)
print(f'The ROC AUC on testing set is: {roc:.3f}')
print(confusion_matrix(y_test_next10k, predictions > 0.5))
print(classification_report(y_test_next10k, predictions > 0.5))

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8
Iteration: 9
Iteration: 10
Iteration: 11
Iteration: 12
Iteration: 13
Iteration: 14
Iteration: 15
Iteration: 16
Iteration: 17
Iteration: 18
Iteration: 19
--- 10.114 seconds ---
The ROC AUC on testing set is: 0.692
[[8069   57]
 [1820   54]]
              precision    recall  f1-score   support

           0       0.82      0.99      0.90      8126
           1       0.49      0.03      0.05      1874

    accuracy                           0.81     10000
   macro avg       0.65      0.51      0.48     10000
weighted avg       0.75      0.81      0.74     10000

